# 📊 **PROYECTO FINAL – IDENTIFICACIÓN DE OPERADORES INEFICACES**

---

## 🎯 **OBJETIVO DEL PROYECTO**

El objetivo de este proyecto es analizar el desempeño de los operadores del servicio **CallMeMaybe** para identificar aquellos con bajo rendimiento operativo.

---

## 🚨 **CRITERIOS DE INEFICIENCIA**

Un operador se considerará **INEFICAZ** si cumple una o más de las siguientes condiciones:

- 🔴 Presenta una alta proporción de llamadas entrantes perdidas  
- ⏳ Tiene tiempos de espera elevados  
- 📉 Realiza pocas llamadas salientes cuando corresponde  

---

## 🔎 **ALCANCE DEL ANÁLISIS**

Para lograrlo, se realizará:

- 📊 Análisis exploratorio de datos  
- 📈 Construcción de métricas de desempeño  
- 🚨 Identificación de operadores ineficaces  
- 🔬 Pruebas estadísticas para validar los hallazgos  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set(style="whitegrid")

In [3]:
calls = pd.read_csv('telecom_dataset_new.csv')
clients = pd.read_csv('telecom_clients.csv')

In [4]:
calls.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25


In [5]:
calls.info()

<class 'pandas.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  str    
 2   direction            53902 non-null  str    
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(1), str(2)
memory usage: 3.3+ MB


In [6]:
calls.describe()

,user_id,operator_id,calls_count,call_duration,total_call_duration
count,53902.000000,45730.000000,53902.000000,53902.000000,53902.000000
mean,167295.344477,916535.993002,16.451245,866.684427,1157.133297
std,598.883775,21254.123136,62.917170,3731.791202,4403.468763
min,166377.000000,879896.000000,1.000000,0.000000,0.000000
25%,166782.000000,900788.000000,1.000000,0.000000,47.000000
50%,167162.000000,913938.000000,4.000000,38.000000,210.000000
75%,167819.000000,937708.000000,12.000000,572.000000,902.000000
max,168606.000000,973286.000000,4817.000000,144395.000000,166155.000000


In [7]:
# Convertir fecha
calls['date'] = pd.to_datetime(calls['date'])

# Revisar valores nulos
calls.isnull().sum()

user_id                   0
date                      0
direction                 0
internal                117
operator_id            8172
is_missed_call            0
calls_count               0
call_duration             0
total_call_duration       0
dtype: int64

In [8]:
calls[calls['operator_id'].isnull()].head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
7,166377,2019-08-05 00:00:00+03:00,in,False,NaN,True,6,0,35
9,166377,2019-08-06 00:00:00+03:00,in,False,NaN,True,4,0,62
17,166377,2019-08-07 00:00:00+03:00,in,False,NaN,True,2,0,24
27,166377,2019-08-12 00:00:00+03:00,in,False,NaN,True,2,0,34


In [9]:
calls[calls['operator_id'].isnull()]['is_missed_call'].value_counts()

is_missed_call
True     8050
False     122
Name: count, dtype: int64

In [10]:
calls_clean = calls.dropna(subset=['operator_id']).copy()

In [11]:
calls_clean.shape

(45730, 9)

In [12]:
calls_clean['waiting_time'] = calls_clean['total_call_duration'] - calls_clean['call_duration']

In [13]:
calls_clean[['call_duration', 'total_call_duration', 'waiting_time']].head()

,call_duration,total_call_duration,waiting_time
1,0,5,5
2,0,1,1
3,10,18,8
4,0,25,25
5,3,29,26


In [14]:
operator_metrics = calls_clean.groupby('operator_id').agg(
    total_calls=('calls_count', 'sum'),
    incoming_calls=('calls_count', lambda x: x[calls_clean.loc[x.index, 'direction'] == 'in'].sum()),
    outgoing_calls=('calls_count', lambda x: x[calls_clean.loc[x.index, 'direction'] == 'out'].sum()),
    missed_calls=('calls_count', lambda x: x[calls_clean.loc[x.index, 'is_missed_call'] == True].sum()),
    avg_waiting_time=('waiting_time', 'mean'),
    avg_call_duration=('call_duration', 'mean')
).reset_index()

In [15]:
operator_metrics.head()

,operator_id,total_calls,incoming_calls,outgoing_calls,missed_calls,avg_waiting_time,avg_call_duration
0,879896.0,1131,60,1071,255,110.671875,650.476562
1,879898.0,7974,118,7856,2594,450.087649,1111.067729
2,880020.0,54,8,46,30,15.181818,104.090909
3,880022.0,219,8,211,118,57.565789,240.842105
4,880026.0,2439,25,2414,696,121.171717,856.939394


In [16]:
operator_metrics['missed_rate'] = operator_metrics['missed_calls'] / operator_metrics['total_calls']

operator_metrics['incoming_ratio'] = operator_metrics['incoming_calls'] / operator_metrics['total_calls']

operator_metrics['outgoing_ratio'] = operator_metrics['outgoing_calls'] / operator_metrics['total_calls']

In [17]:
operator_metrics.head()

,operator_id,total_calls,incoming_calls,outgoing_calls,missed_calls,avg_waiting_time,avg_call_duration,missed_rate,incoming_ratio,outgoing_ratio
0,879896.0,1131,60,1071,255,110.671875,650.476562,0.225464,0.053050,0.946950
1,879898.0,7974,118,7856,2594,450.087649,1111.067729,0.325307,0.014798,0.985202
2,880020.0,54,8,46,30,15.181818,104.090909,0.555556,0.148148,0.851852
3,880022.0,219,8,211,118,57.565789,240.842105,0.538813,0.036530,0.963470
4,880026.0,2439,25,2414,696,121.171717,856.939394,0.285363,0.010250,0.989750


In [18]:
operator_metrics[['missed_rate', 'avg_waiting_time', 'outgoing_calls']].describe()

,missed_rate,avg_waiting_time,outgoing_calls
count,1092.000000,1092.000000,1092.000000
mean,0.290327,215.446189,612.951465
std,0.248426,504.915578,3103.139223
min,0.000000,0.000000,0.000000
25%,0.061557,23.058824,2.000000
50%,0.270196,46.259878,36.500000
75%,0.428571,155.541011,388.750000
max,1.000000,5907.443038,64897.000000


In [19]:
# Definir umbrales
missed_threshold = operator_metrics['missed_rate'].quantile(0.75)
waiting_threshold = operator_metrics['avg_waiting_time'].quantile(0.75)
outgoing_threshold = operator_metrics['outgoing_calls'].quantile(0.25)

# Crear indicadores booleanos
operator_metrics['high_missed'] = operator_metrics['missed_rate'] > missed_threshold
operator_metrics['high_wait'] = operator_metrics['avg_waiting_time'] > waiting_threshold
operator_metrics['low_outgoing'] = operator_metrics['outgoing_calls'] < outgoing_threshold

# Contar cuántas condiciones cumple cada operador
operator_metrics['inefficiency_score'] = (
    operator_metrics['high_missed'].astype(int) +
    operator_metrics['high_wait'].astype(int) +
    operator_metrics['low_outgoing'].astype(int)
)

# Definir ineficientes (2 o más condiciones)
operator_metrics['is_inefficient'] = operator_metrics['inefficiency_score'] >= 2

In [20]:
operator_metrics['is_inefficient'].value_counts()

is_inefficient
False    943
True     149
Name: count, dtype: int64

In [21]:
inefficient = operator_metrics[operator_metrics['is_inefficient'] == True]
efficient = operator_metrics[operator_metrics['is_inefficient'] == False]

inefficient.shape, efficient.shape

((149, 15), (943, 15))

In [22]:
operator_metrics.groupby('is_inefficient')[['missed_rate', 'avg_waiting_time', 'outgoing_calls']].mean()

,missed_rate,avg_waiting_time,outgoing_calls
is_inefficient,,,
False,0.243740,127.027277,437.707317
True,0.585171,775.037020,1722.046980


In [23]:
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(
    inefficient['missed_rate'],
    efficient['missed_rate'],
    equal_var=False
)

t_stat, p_value

(np.float64(19.744386619756604), np.float64(2.483107559058e-50))

## 🔬 **PRUEBA DE HIPÓTESIS – MISSED RATE**

Se realizó una **prueba t de Welch** para comparar la tasa de llamadas perdidas entre operadores eficientes e ineficientes.

---

### 📌 **HIPÓTESIS**

- **H₀:** No existe diferencia en la tasa de llamadas perdidas entre ambos grupos.  
- **H₁:** Existe una diferencia significativa en la tasa de llamadas perdidas.  

---

### 📊 **RESULTADO**

- **p-value ≈ 0.000**
- Se **rechaza H₀**.

---

### 🎯 **CONCLUSIÓN**

Existe evidencia estadísticamente significativa de que los operadores clasificados como **INEFICIENTES** presentan una tasa de llamadas perdidas considerablemente mayor que los operadores eficientes.

In [24]:
t_stat_wait, p_value_wait = ttest_ind(
    inefficient['avg_waiting_time'],
    efficient['avg_waiting_time'],
    equal_var=False
)

t_stat_wait, p_value_wait

(np.float64(8.770319042749664), np.float64(3.0361588297904026e-15))

## 🔬 **PRUEBA DE HIPÓTESIS – WAITING TIME**

Se realizó una **prueba t de Welch** para comparar el tiempo promedio de espera entre operadores eficientes e ineficientes.

---

### 📌 **HIPÓTESIS**

- **H₀:** No existe diferencia en el tiempo promedio de espera entre ambos grupos.  
- **H₁:** Existe una diferencia significativa en el tiempo promedio de espera.  

---

### 📊 **RESULTADO**

- **p-value ≈ 0.000**
- Se **rechaza H₀**.

---

### 🎯 **CONCLUSIÓN**

Existe evidencia estadísticamente significativa de que los operadores clasificados como **INEFICIENTES** presentan tiempos de espera considerablemente mayores que los operadores eficientes.

# 📊 **PROYECTO FINAL – IDENTIFICACIÓN DE OPERADORES INEFICIENTES**

---

## 🧭 **INTRODUCCIÓN**

El objetivo de este proyecto es analizar el desempeño de los operadores del servicio de telefonía virtual **CallMeMaybe** para identificar aquellos con bajo rendimiento operativo.

Un operador se considera **ineficiente** cuando presenta:
- 🔴 Alta tasa de llamadas entrantes perdidas  
- ⏳ Tiempos de espera elevados  
- 📉 Baja productividad en llamadas salientes cuando corresponde  

El análisis incluye exploración de datos, construcción de métricas, validación estadística y recomendaciones de negocio.

---

## 📂 **DESCRIPCIÓN DE LOS DATOS**

### **FUENTES DE INFORMACIÓN**
- Dataset de llamadas (`telecom_dataset`)
- Dataset de clientes (`telecom_clients`)

### **VARIABLES CLAVE**
- `direction` (entrante / saliente)  
- `is_missed_call`  
- `call_duration`  
- `total_call_duration`  
- `operator_id`

---

## 🧼 **PREPARACIÓN Y LIMPIEZA DE DATOS**

### **TRATAMIENTO DE VALORES NULOS**
- Se identificaron llamadas entrantes sin operador asignado.
- Estas llamadas corresponden mayoritariamente a llamadas perdidas antes de ser atendidas.
- ❌ No se consideran para evaluar desempeño individual.

### **CREACIÓN DE VARIABLES**
- ⏱ `waiting_time = total_call_duration - call_duration`

---

## 📊 **ANÁLISIS EXPLORATORIO DE DATOS (EDA)**

### **PATRONES GENERALES**
- Distribución de duración de llamadas
- Relación entre llamadas internas y externas
- Volumen de llamadas por operador

*(Las visualizaciones se presentan en las secciones correspondientes y dashboards)*

---

## 🎯 **CONSTRUCCIÓN DE MÉTRICAS DE DESEMPEÑO**

### **KPIs POR OPERADOR**
- 📞 Total de llamadas
- 📥 Llamadas entrantes
- 📤 Llamadas salientes
- ❌ Llamadas perdidas
- ⏳ Tiempo promedio de espera
- 📊 Tasa de llamadas perdidas (`missed_rate`)

---

## 🚨 **DEFINICIÓN DE OPERADORES INEFICIENTES**

Un operador es clasificado como **ineficiente** si cumple **al menos 2** de las siguientes condiciones:

- 🔴 `missed_rate` > percentil 75  
- ⏳ `avg_waiting_time` > percentil 75  
- 📉 `outgoing_calls` < percentil 25  

Este enfoque está basado en **percentiles**, lo que permite una evaluación relativa y estadísticamente justificada.

---

## 👥 **RESULTADOS DE CLASIFICACIÓN**

### **DISTRIBUCIÓN DE OPERADORES**
- Total de operadores analizados: **1092**
- Operadores eficientes: **943**
- Operadores ineficientes: **149**
- Proporción de ineficiencia: **13.6%**

---

## 📈 **COMPARACIÓN ENTRE GRUPOS**

### **PROMEDIOS POR GRUPO**

| Métrica | Eficientes | Ineficientes |
|-------|------------|--------------|
| 📞 Tasa de llamadas perdidas | 0.24 | 0.58 |
| ⏳ Tiempo promedio de espera | 127 seg | 775 seg |
| 📤 Llamadas salientes | 437 | 1722 |

---

## 🔬 **VALIDACIÓN ESTADÍSTICA**

### **PRUEBA DE HIPÓTESIS – MISSED RATE**

- **H₀:** No existe diferencia entre grupos  
- **H₁:** Existe una diferencia significativa  

📌 Resultado:
- p-value ≈ 0.000  
- ✔ Se rechaza H₀  

---

### **PRUEBA DE HIPÓTESIS – WAITING TIME**

- **H₀:** No existe diferencia entre grupos  
- **H₁:** Existe una diferencia significativa  

📌 Resultado:
- p-value ≈ 0.000  
- ✔ Se rechaza H₀  

---

## 🧠 **INTERPRETACIÓN DE NEGOCIO**

Los operadores ineficientes presentan:
- 🔴 Más del doble de tasa de llamadas perdidas
- ⏳ Tiempos de espera hasta 6 veces mayores
- 📊 Alta variabilidad operativa

Estos patrones sugieren problemas en:
- Gestión del tiempo
- Carga de trabajo
- Procesos internos
- Capacitación

---

## 🚀 **RECOMENDACIONES**

### 📊 **MONITOREO CONTINUO**
Implementar dashboards con:
- Missed rate
- Waiting time
- Productividad por operador

### 🎓 **CAPACITACIÓN DIRIGIDA**
Entrenamiento específico para operadores con métricas críticas.

### ⚖ **BALANCE DE CARGA**
Reasignar volumen de llamadas para evitar sobrecarga.

### 🔁 **SEGUIMIENTO PERIÓDICO**
Evaluación mensual usando esta metodología.

---

## 🎯 **IMPACTO ESPERADO**

- 📉 Reducción de llamadas perdidas  
- ⭐ Mejora en satisfacción del cliente  
- 📈 Mayor eficiencia operativa  

---

## 📌 **CONCLUSIÓN FINAL**

La metodología propuesta permite identificar de forma objetiva y estadísticamente sólida a los operadores con bajo desempeño, brindando una herramienta accionable para la toma de decisiones operativas y estratégicas.